# Structure Refinement: HS, HRPT

This example demonstrates a Rietveld refinement of HS crystal structure
using constant wavelength neutron powder diffraction data from HRPT at PSI.

## Import Library

In [ ]:
# Check if the easydiffraction library is installed.
# If not, install it including the 'visualization' extras.
# This is needed, e.g., when running this as a notebook via Google Colab.
import builtins
import importlib.util

if hasattr(builtins, '__IPYTHON__'):
    if importlib.util.find_spec('easydiffraction') is None:
        print('Installing the easydiffraction library...')
        !pip install 'easydiffraction[visualization]'

In [ ]:
from easydiffraction import Experiment
from easydiffraction import Project
from easydiffraction import SampleModel
from easydiffraction import download_from_repository

## Define Sample Model

This section shows how to add sample models and modify their parameters.

#### Create Sample Model

In [ ]:
model = SampleModel('hs')

#### Set Space Group

In [ ]:
model.space_group.name_h_m = 'R -3 m'
model.space_group.it_coordinate_system_code = 'h'

#### Set Unit Cell

In [ ]:
model.cell.length_a = 6.9
model.cell.length_c = 14.1

#### Set Atom Sites

In [ ]:
model.atom_sites.add('Zn', 'Zn', 0, 0, 0.5, wyckoff_letter='b', b_iso=0.5)
model.atom_sites.add('Cu', 'Cu', 0.5, 0, 0, wyckoff_letter='e', b_iso=0.5)
model.atom_sites.add('O', 'O', 0.21, -0.21, 0.06, wyckoff_letter='h', b_iso=0.5)
model.atom_sites.add('Cl', 'Cl', 0, 0, 0.197, wyckoff_letter='c', b_iso=0.5)
model.atom_sites.add('H', '2H', 0.13, -0.13, 0.08, wyckoff_letter='h', b_iso=0.5)

#### Symmetry constraints

Show CIF output before applying symmetry constraints.

In [ ]:
model.show_as_cif()

Apply symmetry constraints.

In [ ]:
model.apply_symmetry_constraints()

Show CIF output after applying symmetry constraints.

In [ ]:
model.show_as_cif()

## Define Experiment

This section shows how to add experiments, configure their parameters, and
link the sample models defined in the previous step.

#### Download Measured Data

In [ ]:
download_from_repository('hrpt_hs.xye', destination='data')

#### Create Experiment

In [ ]:
expt = Experiment(name='hrpt', data_path='data/hrpt_hs.xye')

#### Set Instrument

In [ ]:
expt.instrument.setup_wavelength = 1.89
expt.instrument.calib_twotheta_offset = 0.0

#### Set Peak Profile

In [ ]:
expt.peak.broad_gauss_u = 0.1
expt.peak.broad_gauss_v = -0.2
expt.peak.broad_gauss_w = 0.2
expt.peak.broad_lorentz_x = 0.0
expt.peak.broad_lorentz_y = 0

#### Set Background

In [ ]:
expt.background.add(x=4.4196, y=500)
expt.background.add(x=6.6207, y=500)
expt.background.add(x=10.4918, y=500)
expt.background.add(x=15.4634, y=500)
expt.background.add(x=45.6041, y=500)
expt.background.add(x=74.6844, y=500)
expt.background.add(x=103.4187, y=500)
expt.background.add(x=121.6311, y=500)
expt.background.add(x=159.4116, y=500)

#### Set Linked Phases

In [ ]:
expt.linked_phases.add('hs', scale=0.5)

## Define Project

The project object is used to manage the sample model, experiment, and
analysis.

#### Create Project

In [ ]:
project = Project()

#### Set Plotting Engine

In [ ]:
project.plotter.engine = 'plotly'

#### Add Sample Model

In [ ]:
project.sample_models.add(model)

#### Add Experiment

In [ ]:
project.experiments.add(expt)

## Perform Analysis

This section shows the analysis process, including how to set up
calculation and fitting engines.

#### Set Calculator

In [ ]:
project.analysis.current_calculator = 'cryspy'

#### Set Minimizer

In [ ]:
project.analysis.current_minimizer = 'lmfit (leastsq)'

#### Plot Measured vs Calculated

In [ ]:
project.plot_meas_vs_calc(expt_name='hrpt', show_residual=True)

In [ ]:
project.plot_meas_vs_calc(expt_name='hrpt', x_min=48, x_max=51, show_residual=True)

### Perform Fit 1/5

Set parameters to be refined.

In [ ]:
model.cell.length_a.free = True
model.cell.length_c.free = True

expt.linked_phases['hs'].scale.free = True
expt.instrument.calib_twotheta_offset.free = True

Show free parameters after selection.

In [ ]:
project.analysis.show_free_params()

#### Run Fitting

In [ ]:
project.analysis.fit()

#### Plot Measured vs Calculated

In [ ]:
project.plot_meas_vs_calc(expt_name='hrpt', show_residual=True)

In [ ]:
project.plot_meas_vs_calc(expt_name='hrpt', x_min=48, x_max=51, show_residual=True)

### Perform Fit 2/5

Set more parameters to be refined.

In [ ]:
expt.peak.broad_gauss_u.free = True
expt.peak.broad_gauss_v.free = True
expt.peak.broad_gauss_w.free = True
expt.peak.broad_lorentz_x.free = True

for point in expt.background:
    point.y.free = True

Show free parameters after selection.

In [ ]:
project.analysis.show_free_params()

#### Run Fitting

In [ ]:
project.analysis.fit()

#### Plot Measured vs Calculated

In [ ]:
project.plot_meas_vs_calc(expt_name='hrpt', show_residual=True)

In [ ]:
project.plot_meas_vs_calc(expt_name='hrpt', x_min=48, x_max=51, show_residual=True)

### Perform Fit 3/5

Set more parameters to be refined.

In [ ]:
model.atom_sites['O'].fract_x.free = True
model.atom_sites['O'].fract_z.free = True
model.atom_sites['Cl'].fract_z.free = True
model.atom_sites['H'].fract_x.free = True
model.atom_sites['H'].fract_z.free = True

Show free parameters after selection.

In [ ]:
project.analysis.show_free_params()

#### Run Fitting

In [ ]:
project.analysis.fit()

#### Plot Measured vs Calculated

In [ ]:
project.plot_meas_vs_calc(expt_name='hrpt', show_residual=True)

In [ ]:
project.plot_meas_vs_calc(expt_name='hrpt', x_min=48, x_max=51, show_residual=True)

### Perform Fit 4/5

Set more parameters to be refined.

In [ ]:
model.atom_sites['Zn'].b_iso.free = True
model.atom_sites['Cu'].b_iso.free = True
model.atom_sites['O'].b_iso.free = True
model.atom_sites['Cl'].b_iso.free = True
model.atom_sites['H'].b_iso.free = True

Show free parameters after selection.

In [ ]:
project.analysis.show_free_params()

#### Run Fitting

In [ ]:
project.analysis.fit()

#### Plot Measured vs Calculated

In [ ]:
project.plot_meas_vs_calc(expt_name='hrpt', show_residual=True)

In [ ]:
project.plot_meas_vs_calc(expt_name='hrpt', x_min=48, x_max=51, show_residual=True)

## Summary

This final section shows how to review the results of the analysis.

#### Show Project Summary

In [ ]:
project.summary.show_report()